# Version III - Massive data augmentation

### Imports and initial setup

In [1]:
import import_ipynb

#pip install tensorflow-addons
import tensorflow_addons as tfa

from InitialSetup import dataset, BATCH_SIZE, tf, prepare_callbacks, model_VI, show_history, test_dataset, AUTOTUNE, process_brightness, process_contrast, process_hue, process_saturation, process_rotate, process_shear, process_translate, process_crop

importing Jupyter notebook from InitialSetup.ipynb
Num GPUs Available:  1
Total images in validatation dataset:  12630
(32, 32, 3) (43,)
Total images in dataset:  39209


In [2]:
dataSoloV3 = dataset
# color ops
dataSoloV3 = dataSoloV3.map(process_brightness)
dataSoloV3 = dataSoloV3.concatenate(dataset.map(process_contrast))
dataSoloV3 = dataSoloV3.concatenate(dataset.map(process_hue))
dataSoloV3 = dataSoloV3.concatenate(dataset.map(process_saturation))

#geometry ops
dataSoloV3 = dataSoloV3.concatenate(dataset.map(process_rotate))
dataSoloV3 = dataSoloV3.concatenate(dataset.map(process_shear))
dataSoloV3 = dataSoloV3.concatenate(dataset.map(process_translate))
dataSoloV3 = dataSoloV3.concatenate(dataset.map(process_crop))

dataSoloV3_size = tf.data.experimental.cardinality(dataSoloV3).numpy()

dataSoloV3 = dataSoloV3.cache()
dataSoloV3 = dataSoloV3.shuffle(buffer_size = dataSoloV3_size)
dataSoloV3 = dataSoloV3.batch(batch_size = BATCH_SIZE)
dataSoloV3 = dataSoloV3.prefetch(buffer_size = AUTOTUNE)
dataSoloV3 = dataSoloV3.repeat()

train_size = int(0.8* dataSoloV3_size)
val_size = int(0.2* dataSoloV3_size)

train_dataset = dataSoloV3.take(train_size)
val_dataset = dataSoloV3.skip(train_size)

### Setup model and save place

In [3]:
modelV3 = model_VI(43, 32, 3)

file_pathV3 = './Networks/MassiveDataAugmentation.ckpt'

callbacksV3 = prepare_callbacks(file_pathV3)

### Train model

In [5]:
historyV3 = modelV3.fit(train_dataset, steps_per_epoch = train_size/BATCH_SIZE,
          epochs=20, 
          validation_data = val_dataset, 
          validation_steps = val_size/BATCH_SIZE,
          callbacks = callbacksV3)

Epoch 1/20
3920/3920 [==============================] - 190s 30ms/step - loss: 1.8308 - accuracy: 0.5210 - val_loss: 0.1399 - val_accuracy: 0.9635

Epoch 00001: val_accuracy improved from -inf to 0.96351, saving model to ./Networks\MassiveDataAugmentation.ckpt
Epoch 2/20
3920/3920 [==============================] - 117s 30ms/step - loss: 0.2260 - accuracy: 0.9343 - val_loss: 0.0525 - val_accuracy: 0.9860

Epoch 00002: val_accuracy improved from 0.96351 to 0.98603, saving model to ./Networks\MassiveDataAugmentation.ckpt
Epoch 3/20
3920/3920 [==============================] - 118s 30ms/step - loss: 0.1150 - accuracy: 0.9658 - val_loss: 0.0398 - val_accuracy: 0.9903

Epoch 00003: val_accuracy improved from 0.98603 to 0.99025, saving model to ./Networks\MassiveDataAugmentation.ckpt
Epoch 4/20
3920/3920 [==============================] - 117s 30ms/step - loss: 0.0805 - accuracy: 0.9761 - val_loss: 0.0218 - val_accuracy: 0.9944

Epoch 00004: val_accuracy improved from 0.99025 to 0.99439, sav

### Show the learning history

In [ ]:
show_history(historyV3)

### Load weights from file and test with validation and test dataset

In [5]:
modelV3.load_weights(file_pathV3)

evalV3 = modelV3.evaluate(test_dataset, verbose=2)
valV3 = modelV3.evaluate(val_dataset, steps=val_size/BATCH_SIZE, verbose=2)

198/198 - 4s - loss: 0.0262 - accuracy: 0.9922
980/980 - 121s - loss: 0.0108 - accuracy: 0.9973
